In [1]:
import pandas as pd
import glob
import json
import numpy as np
from pandasql import sqldf

In [2]:
pysqldf = lambda q: sqldf(q, globals())

In [3]:
files = glob.glob('output-idl-json-files.archive.org/download/**/*.json', recursive=True)

In [ ]:
#len(files)

In [ ]:
#files[0]

In [ ]:
#f = open(files[0])
#data = json.load(f) 

In [ ]:
#data.keys()

In [ ]:
#data['annotation_results'][0].keys()

In [ ]:
#data['annotation_results'][1].keys()

In [ ]:
#data['annotation_results'][0]['input_uri']

In [ ]:
#data['annotation_results'][0]['segment_label_annotations'][0]['entity']['description']

In [ ]:
#data['annotation_results'][0]['shot_label_annotations'][0]['entity']['description']

In [ ]:
#data['annotation_results'][0]['text_annotations'][0]['text']

In [ ]:
#data['annotation_results'][1]['speech_transcriptions'][2]['alternatives'][0]['transcript']

In [ ]:
#data['annotation_results'][1]['speech_transcriptions'][2]['alternatives'][0]['confidence']

In [5]:
label_annotations = []
text_annotations = []
logo_annotations = []
speech_annotations = []

for file in files:
    
    f = open(file)
    data = json.load(f) 
    
    for i in range(len(data['annotation_results'])):
    
        ar = data['annotation_results'][i]

        uri = ar['input_uri']

        if 'shot_label_annotations' in ar:

            slas = []
            for sla in ar['shot_label_annotations']:
                label = sla['entity']['description']
                slas.append(label)

            label_annotations.append((uri, ','.join(slas)))

        if 'text_annotations' in ar:
            tas = []
            for ta in ar['text_annotations']:
                text = ta['text']
                tas.append(text)

            text_annotations.append((uri, ' '.join(tas)))

        if 'logo_recognition_annotations' in ar.keys():                         
            lras = []
            for lra in ar['logo_recognition_annotations']:                  
                logo = lra['entity']['description']
                lras.append(logo)

            logo_annotations.append((uri, ','.join(lras)))


        #['speech_transcriptions'][2]['alternatives'][0]['transcript']
        if 'speech_transcriptions' in ar:

            sts = []
            scs = []

            for st in ar['speech_transcriptions']:

                alt_conf = st['alternatives'][0]

                if 'transcript' in alt_conf:
                    speech = alt_conf['transcript']
                    speech_confidence = alt_conf['confidence']
                    #print(speech_confidence)
                    sts.append(speech)
                    if speech_confidence == []:
                        speech_confidence = 0
                        
                    scs.append(float(speech_confidence))

            speech_annotations.append((uri, ''.join(sts), np.mean(scs)))
        
    #print(file)

0.83336419
0.82094657
0.91283858
0.84687734
0.8885408
0.73019511
0.91283858
0.91283858
0.53957826
0.91283858
0.91283864
0.90464371
0.86440736
0.73875564
0.87558585
0.89909852
0.87485695
0.89422107
0.7379747
0.82910407
0.89397383
0.91283852
0.91283858
0.75229806
0.78277797
0.84811664
0.84612954
0.79810578
0.91011274
0.91283864
0.81318116
0.90947419
0.84439731
0.87982351
0.83947146
0.76106322
0.75800502
0.85517496
0.81408358
0.91283864
0.91283858
0.61144912
0.64751351
0.62533998
0.83107883
0.8208558
0.77090889
0.6552633
0.778611
0.72275972
0.91283864
0.91283864
0.91283858
0.72106892
0.91283864
0.81340384
0.91283864
0.74964601
0.91283852
0.85239536
0.75113112
0.78038961
0.91283852
0.72698319
0.6999405
0.90879172
0.80369353
0.81683517
0.91283858
0.78031576
0.89470613
0.80701274
0.80070335
0.91283852
0.84161758
0.911273
0.91283858
0.91283852
0.91283852
0.62990594
0.74750763
0.85041821
0.91283864
0.84275544
0.68552369
0.8768875
0.76659286
0.87446636
0.91283858
0.77910823
0.72765094
0.8876735

0.82877201
0.8961519
0.81950808
0.80402243
0.9128387
0.87375712
0.91283822
0.9128387
0.91283828
0.87555277
0.8436029
0.77451521
0.73397791
0.88134933
0.89432037
0.91187465
0.82248759
0.91283828
0.91283822
0.87068826
0.91283828
0.91280597
0.91283858
0.85736948
0.90085721
0.85664231
0.87623298
0.86993283
0.83735752
0.81589764
0.78357315
0.86998856
0.91283858
0.91283852
0.91038686
0.77608418
0.8724978
0.91220295
0.9124245
0.8549912
0.89714175
0.9128387
0.84356147
0.9080708
0.84021831
0.85646844
0.91283852
0.88214898
0.85266078
0.90526247
0.9048143
0.89343101
0.81919539
0.85376358
0.91283858
0.67273027
0.84410685
0.77561086
0.91283858
0.81536055
0.91283882
0.81206805
0.90459198
0.89327383
0.91283864
0.88978016
0.72522813
0.80419314
0.75724959
0.80343497
0.91283864
0.88093489
0.80477524
0.84071726
0.81549579
0.76095265
0.87375963
0.84126747
0.81194836
0.91283864
0.91283834
0.88300776
0.91283858
0.83771372
0.88670868
0.69369954
0.90010035
0.91283858
0.9128387
0.91283864
0.91283822
0.81735653

0.79698914
0.75902259
0.71302754
0.78780669
0.90129006
0.7594074
0.79762077
0.77687109
0.84622687
0.78756315
0.73751938
0.77522635
0.84276181
0.91283834
0.91283822
0.83734864
0.91234255
0.87162077
0.89018947
0.91283858
0.72927839
0.83230633
0.74659932
0.76784551
0.87642705
0.91283834
0.91283858
0.88812655
0.89492834
0.88455379
0.88397962
0.89242524
0.8294608
0.9026807
0.90793324
0.73375338
0.88834274
0.89803088
0.86118639
0.6706723
0.89726591
0.82167625
0.77302063
0.85132021
0.80601692
0.79664463
0.88826352
0.86819631
0.91283864
0.84358436
0.78775537
0.83417219
0.91283858
0.88254398
0.91283828
0.89920139
0.88694435
0.91283828
0.79447335
0.88001192
0.75348717
0.79274029
0.91283834
0.91283828
0.77631485
0.84600735
0.77843243
0.88978857
0.89435273
0.89015943
0.7975443
0.86727387
0.91283828
0.76218522
0.91283876
0.90512627
0.90083748
0.79020101
0.89242822
0.86055976
0.90730739
0.83116055
0.8514722
0.8645432
0.91283858
0.91283858
0.91283858
0.91283858
0.91283864
0.91283858
0.84207475
0.8548

0.79923236
0.76141119
0.85765058
0.84920967
0.8762567
0.78727579
0.72638935
0.78512806
0.9128387
0.90106332
0.91270286
0.8063584
0.82148635
0.91283858
0.82146859
0.91283834
0.91283828
0.91283828
0.70449501
0.78302515
0.88029116
0.74835718
0.81608337
0.80207646
0.84347188
0.81887031
0.80341345
0.75897396
0.79528779
0.83317548
0.81102806
0.91283858
0.91283834
0.91283846
0.75635815
0.91283858
0.65862781
0.91283834
0.88368493
0.91283858
0.88349319
0.72804326
0.82465124
0.91283852
0.8285017
0.81394821
0.86814982
0.73119068
0.85913759
0.89255387
0.91283858
0.91283858
0.82881069
0.79933691
0.84923923
0.9128387
0.88148654
0.77339137
0.88671637
0.9125244
0.8319636
0.9033308
0.9128387
0.88188326
0.80082548
0.77407521
0.79014093
0.91283834
0.87199146
0.83634937
0.78599328
0.78682399
0.9128387
0.91283852
0.9128387
0.90703344
0.80977952
0.76844311
0.87730026
0.8767947
0.89970779
0.87661552
0.91283828
0.91283858
0.91283864
0.86464238
0.77477461
0.9027136
0.65217143
0.9128387
0.90770549
0.8696118
0.9

0.78980929
0.81161016
0.72284162
0.90043265
0.84319657
0.81670076
0.91283858
0.86647046
0.85402936
0.75818694
0.91283864
0.91211623
0.79085118
0.91283858
0.67411613
0.90320081
0.87010628
0.8112154
0.8241722
0.86050606
0.74465227
0.88235319
0.82319927
0.85967499
0.74345058
0.85484332
0.91283858
0.84202832
0.80711746
0.70501006
0.91283858
0.7753762
0.91283852
0.89321965
0.81221515
0.84823495
0.74116075
0.87636805
0.84416503
0.75593388
0.78814936
0.82304722
0.77163404
0.85029489
0.90329522
0.91283834
0.91283858
0.91065162
0.8882333
0.78496188
0.74519205
0.85295147
0.91283834
0.91283828
0.91283828
0.9128384
0.91283852
0.91283858
0.8085193
0.91283846
0.76630431
0.85636634
0.79511523
0.91283852
0.78832036
0.89236039
0.83886701
0.67941064
0.81803226
0.91283864
0.89929116
0.80863452
0.91283834
0.78467506
0.91017777
0.75002635
0.7979331
0.91283828
0.91283828
0.71251667
0.85446453
0.87531865
0.84860373
0.91283858
0.84152478
0.91283858
0.90898812
0.84022516
0.91283828
0.91283822
0.79131168
0.9128

...

0.7426542
0.76554972
0.91157126
0.81861693
0.83525002
0.83876926
0.91283858
0.86219585
0.77528787
0.91283858
0.80430627
0.91283858
0.84995657
0.8562296
0.84975249
0.81735086
0.91283858
0.91283864
0.91283864
0.91283852
0.84843808
0.9128384
0.82148105
0.91283852
0.9128387
0.91283852
0.84543914
0.79286909
0.91283858
0.90914875
0.838902
0.91283858
0.9128387
0.72132957
0.91283822
0.7322287
0.83000058
0.91048819
0.82131839
0.85409206
0.91283852
0.75703824
0.9128387
0.85270452
0.91283858
0.90468949
0.85559839
0.88923168
0.91283858
0.80354553
0.91283858
0.89540511
0.78450239
0.91283852
0.82187551
0.88207144
0.89591771
0.78421175
0.88425624
0.82417536
0.9011001
0.8588441
0.89162534
0.9128384
0.91283852
0.90217566
0.9128384
0.83119452
0.78387451
0.78689647
0.9128387
0.80734891
0.8056401
0.87518126
0.88760155
0.84750527
0.91283852
0.89844733
0.91283852
0.74857783
0.87156683
0.68793607
0.90000385
0.78733152
0.80241936
0.73558807
0.887407
0.91283852
0.82801181
0.85305268
0.91283822
0.86749047
0.851

0.8695423
0.91283858
0.7557351
0.73252803
0.91283828
0.75939798
0.9128387
0.88409096
0.89633298
0.91153395
0.91283822
0.9128387
0.83390474
0.90942985
0.81726664
0.91283852
0.77171832
0.75967497
0.89148742
0.8953222
0.91283864
0.87329757
0.91283828
0.91283822
0.86054301
0.89117461
0.73784643
0.86327869
0.8384589
0.9128387
0.90107155
0.9078564
0.91283858
0.89447808
0.82032722
0.84014034
0.80435425
0.61360902
0.80043507
0.89414716
0.9128387
0.91283864
0.85932618
0.82630152
0.90803713
0.90718639
0.91283834
0.91283858
0.76187742
0.88734055
0.81751114
0.9128384
0.90847081
0.84008324
0.84186929
0.78581411
0.89287686
0.91283846
0.85504556
0.81810987
0.78997141
0.90165079
0.8179214
0.84475738
0.77661359
0.85951996
0.90274739
0.88491189
0.88522035
0.8623392
0.84718227
0.62667024
0.80613285
0.6281178
0.91283858
0.83621824
0.817114
0.90129751
0.87501675
0.87308383
0.47222674
0.87931556
0.91189373
0.64864069
0.7211436
0.86044878
0.9128384
0.83155733
0.89916527
0.82441503
0.87146384
0.84685409
0.894

0.7609188
0.86032033
0.71401775
0.91283852
0.78734243
0.78210622
0.84654677
0.91042268
0.80784011
0.68002903
0.86569613
0.75607771
0.91208148
0.87660033
0.88822442
0.88152528
0.79828328
0.79987609
0.9116711
0.71427345
0.79388529
0.89257795
0.85127896
0.82319403
0.8076508
0.91283929
0.84022951
0.86147267
0.83194995
0.83969051
0.88334829
0.83651328
0.8861267
0.85252553
0.90406889
0.89991635
0.9010061
0.91283858
0.91283858
0.85964888
0.87018526
0.91283864
0.8126815
0.9128387
0.85887605
0.90771997
0.83134657
0.86298549
0.89577484
0.71234089
0.80367154
0.85547638
0.82398373
0.86790663
0.91283828
0.85365528
0.79535812
0.76650721
0.91283864
0.80103439
0.77321845
0.91283858
0.9128387
0.88233769
0.8817758
0.91218024
0.91233033
0.88920259
0.89149696
0.90203762
0.87236452
0.82300109
0.8771919
0.91283834
0.9128387
0.77985239
0.86184752
0.81201625
0.70730639
0.88528097
0.81058717
0.86191714
0.91283852
0.89412755
0.88905722
0.8703568
0.89874947
0.87239015
0.81421083
0.80001557
0.86575192
0.9128387
0

0.83616769
0.7537213
0.77414411
0.86371565
0.78838617
0.69713384
0.81434536
0.90349072
0.85126323
0.89396775
0.84913075
0.81550062
0.75835371
0.81336629
0.9034304
0.91283876
0.91283828
0.75546783
0.91283822
0.86286026
0.87532467
0.87026936
0.91283852
0.87418294
0.87152886
0.79863948
0.81932861
0.85774356
0.84585291
0.89550579
0.83040547
0.79092354
0.73028225
0.91215056
0.81769311
0.78080958
0.79360116
0.88219225
0.89931166
0.798747
0.87084448
0.82754034
0.91067749
0.76197785
0.91283888
0.83236271
0.79943007
0.73297167
0.81274641
0.79075491
0.73608673
0.8911317
0.79112655
0.82051194
0.85465896
0.85054338
0.73427576
0.83015627
0.9128387
0.86730731
0.81363875
0.88649911
0.84190106
0.90604687
0.88933474
0.77185631
0.71946049
0.80318993
0.75974828
0.75836766
0.85546023
0.84485376
0.77801585
0.76692575
0.82333493
0.76943576
0.76474625
0.82508093
0.91283834
0.91062587
0.70963627
0.87687337
0.88306934
0.90142018
0.83724755
0.79005206
0.89930427
0.80421156
0.79227018
0.80212539
0.90966856
0.896

0.91283858
0.88631785
0.7422294
0.7228725
0.84637558
0.8575893
0.91283858
0.78895122
0.9101463
0.73872173
0.91140568
0.91283864
0.75218493
0.85108358
0.83770722
0.85076022
0.64439625
0.89215243
0.91283834
0.71693331
0.83227342
0.90282112
0.86442816
0.91283858
0.82648432
0.87756246
0.84651387
0.83598852
0.90995646
0.82558703
0.84987283
0.86271799
0.76819694
0.73777771
0.85506678
0.91063428
0.85665405
0.91283852
0.90511948
0.91068989
0.88769406
0.86780536
0.91283852
0.91283858
0.84158975
0.91283858
0.88796949
0.76298797
0.91283852
0.91283858
0.48443019
0.87729913
0.91283864
0.65550566
0.82803386
0.91283828
0.83302945
0.83165711
0.87131298
0.91283858
0.88535309
0.90060008
0.8812322
0.63607305
0.89680481
0.91283822
0.884359
0.90041322
0.84448129
0.89839721
0.85099083
0.88977605
0.9128387
0.88347441
0.7795046
0.83684534
0.90294635
0.78497678
0.79218
0.91283852
0.91283858
0.86241609
0.84225601
0.75181741
0.91283864
0.90078568
0.8401072
0.86970598
0.72326809
0.91283852
0.90789622
0.91283858
0

In [33]:
df_label_annotations = pd.DataFrame(label_annotations, columns=['uri', 'label'])
df_text_annotations = pd.DataFrame(text_annotations, columns=['uri', 'text'])
df_logo_annotations = pd.DataFrame(logo_annotations, columns=['uri', 'logo'])
df_speech_annotations = pd.DataFrame(speech_annotations, columns=['uri', 'transcript', 'confidence'])

In [41]:
df_annotations = pysqldf("""
SELECT 
    sa.uri,
    sa.transcript,
    sa.confidence,
    ta.text,
    la.label,
    lga.logo
FROM
    df_speech_annotations sa
LEFT JOIN
    df_label_annotations la
ON
    sa.uri = la.uri
LEFT JOIN
    df_text_annotations ta
ON
    sa.uri = ta.uri
LEFT JOIN
    df_logo_annotations lga
ON
    sa.uri = lga.uri
""")

In [42]:
df_annotations.insert(0, 'file_name', df_annotations['uri'].str.split('/').str[-1][:-4])

In [43]:
df_annotations.insert(0, 'identifier', df_annotations['uri'].str.split('/').str[-2])

In [44]:
df_annotations.head(2)

,identifier,file_name,uri,transcript,confidence,text,label,logo
0,tobacco_qno71d00,VTS_01_1_512kb.mp4,/input-idl-video-files/archive.org/download/to...,This is tape number two of the mr. Edward Blac...,0.836341,PM12 06 PM12:16 PM12:04 (MAY. 19 1998 PM12 10 ...,"speech,professional,official,spokesperson,text...",None
1,tobacco_kon79e00,VTS_01_1_512kb.mp4,/input-idl-video-files/archive.org/download/to...,Good morning. rather crucial most of you have ...,0.835256,PLAY 0:0000 0:00-00 0:00-90 PLEASE PUSH REC KE...,"text,electric blue,sports,presentation,perform...","Green Bay Packers,Renault"


In [45]:
df_annotations.to_csv('csv_files/transcript_annotations.csv', index=False)